# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234709570286                   -0.50    8.0         
  2   -7.250200535703       -1.81       -1.40    1.0   5.01ms
  3   -7.251239754851       -2.98       -2.08    2.0   6.08ms
  4   -7.251224029957   +   -4.80       -2.11    2.0   6.35ms
  5   -7.251324021740       -4.00       -2.58    1.0   5.16ms
  6   -7.251338018207       -4.85       -3.18    1.0   5.29ms
  7   -7.251338693130       -6.17       -3.55    2.0   6.55ms
  8   -7.251338781646       -7.05       -3.98    1.0   5.81ms
  9   -7.251338797547       -7.80       -4.56    2.0   6.77ms
 10   -7.251338798580       -8.99       -5.09    1.0   5.78ms
 11   -7.251338798686       -9.97       -5.43    2.0   7.11ms
 12   -7.251338798702      -10.80       -5.94    1.0   6.01ms
 13   -7.251338798704      -11.81       -6.16    2.0   7.16ms
 14   -7.251338798704      -12.26       -6.67    1.0   6.13ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05214909378950525
[ Info: Arnoldi iteration step 2: normres = 0.6466395595562322
[ Info: Arnoldi iteration step 3: normres = 0.8576247088236325
[ Info: Arnoldi iteration step 4: normres = 0.3178521262913532
[ Info: Arnoldi iteration step 5: normres = 0.4585354026504468
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (4.32e-02, 7.99e-02, 3.47e-01, 2.84e-01, 2.12e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3567936907679814
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.28e-02, 1.26e-01, 3.04e-01, 9.75e-02, 9.58e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08296660266298259
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.44e-04, 1.50e-02, 1.89e-02, 3.77e-02, 5.74e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11648449120042077
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.79e-0